# Optimization Of Facility Location In Zürich

Below is an optimization model that finds the best location to place an Amazon warehouse in Zürich. Details about the assumptions taken when building the model are discussed in the report.

## Installation of required libraries

In [1]:
!pip install pandas
!pip install numpy
!pip install ortools
!pip install geopy

In [2]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model as cp
from geopy.distance import geodesic as GD

## Data loading

Longtidue and latitude for each of the postal codes in Zürich.

Source:	https://www.geonames.org/postal-codes/CH/ZH/zurich.html

In [3]:
lon_lat = pd.read_csv('postal_codes_lon_lat.csv')
lon_lat

,Postal Code,Lattitude,Longitude
0,8001,47.372,8.542
1,8002,47.36,8.533
2,8003,47.373,8.514
3,8004,47.379,8.520
4,8005,47.388,8.521
5,8006,47.387,8.550
6,8008,47.354,8.559
7,8032,47.368,8.566
8,8037,47.398,8.524
9,8038,47.342,8.535


Population of each of the postal codes in Zürich

Source: https://postal-codes.cybo.com/switzerland/zürich/#listcodes

In [4]:
population = pd.read_csv('postal_codes_population.csv')
population

,Postal Code,City,Administrative Region,Population,Area
0,8000,Zürich,Canton of Zürich,—,—
1,8001,Zürich,Canton of Zürich,14.094,1.86 km²
2,8002,Zürich,Canton of Zürich,12.850,2.847 km²
3,8003,Zürich,Canton of Zürich,10.317,1.443 km²
4,8004,Zürich,Canton of Zürich,18.587,2.326 km²
...,...,...,...,...,...
61,8092,Zürich,Canton of Zürich,—,—
62,8093,Zürich,Canton of Zürich,—,—
63,8096,Zürich,Canton of Zürich,—,—
64,8098,Zürich,Canton of Zürich,—,—


The postal codes without an area and population cannot be used for modeling, and will therefore later be removed.

## Data Preprocessing

In [5]:
#Drop redundant columns
population=population[['Postal Code', 'Population']]

#Remove the . in the 'Population'-column splitting each 000
population['Population']=population['Population'].str.replace('.', '', regex=True)

#Merge the population and lon_lat dataframes. Thus also the redundant rows are removed
postal_codes=pd.merge(lon_lat, population, on='Postal Code')

#Change the type of postal Code to String
postal_codes=postal_codes.astype({'Postal Code': 'int'})

#Remove the symbols that show up for some reason
postal_codes['Lattitude']=postal_codes['Lattitude'].str.replace('\xa0', '')

postal_codes

/var/folders/vh/q3_qjqld15g8wpp196t90sym0000gn/T/ipykernel_94854/1374345061.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['Population']=population['Population'].str.replace('.', '', regex=True)


,Postal Code,Lattitude,Longitude,Population
0,8001,47.372,8.542,14094
1,8002,47.36,8.533,12850
2,8003,47.373,8.514,10317
3,8004,47.379,8.520,18587
4,8005,47.388,8.521,17072
5,8006,47.387,8.550,14198
6,8008,47.354,8.559,15911
7,8032,47.368,8.566,15764
8,8037,47.398,8.524,10129
9,8038,47.342,8.535,16995


In [6]:
#Create a list of longitudes and latitudes to be used when calculating distances
lon_lat_list=postal_codes[['Lattitude','Longitude']].values.tolist()

#Create a two-dimensional list with the distances between each postal code
a=[]
for i in range(len(postal_codes)):
  b=[]
  for j in range(len(postal_codes)):
    location_from=lon_lat_list[i]
    location_to=lon_lat_list[j]
    b.append(int(GD(location_from, location_to).m))
  a.append(b)

#Create a dataframe with the distances
distances=pd.DataFrame(a)
distances_with_labels=pd.DataFrame(a,  columns=postal_codes['Postal Code'].unique(), index=postal_codes['Postal Code'].unique())
distances_with_labels

,8001,8002,8003,8004,8005,8006,8008,8032,8037,8038,8045,8046,8047,8048,8049,8050,8051,8052,8055,8057
8001,0,1497,2117,1834,2383,1773,2377,1866,3194,3376,3267,6091,4235,5005,5482,4498,4217,5783,3733,3121
8002,1497,0,2036,2329,3242,3264,2074,2646,4279,2006,1973,7127,3876,4998,6192,5938,5685,7135,2979,4538
8003,2117,2036,0,806,1749,3132,4002,3966,2880,3794,2255,5460,2117,3012,4277,5157,5748,6000,1811,3806
8004,1834,2329,806,0,1003,2433,4050,3683,2133,4266,3007,4852,2627,3192,3870,4351,5015,5225,2576,3002
8005,2383,3242,1749,1003,0,2192,4746,4061,1134,5222,3995,3887,3067,3178,3099,3499,4539,4251,3358,2250
8006,1773,3264,3132,2433,2192,0,3731,2433,2312,5129,4886,4971,5044,5362,4884,2780,2625,4182,4942,1493
8008,2377,2074,4002,4050,4746,3731,0,1643,5560,2251,3779,8450,5945,7011,7841,6476,5418,7913,5034,5222
8032,1866,2646,3966,3683,4061,2433,1643,0,4602,3720,4617,7386,6078,6868,7094,5021,3779,6528,5456,3895
8037,3194,4279,2880,2133,1134,2312,5560,4602,0,6281,5129,2898,3918,3649,2573,2564,4089,3132,4406,1600
8038,3376,2006,3794,4266,5222,5129,2251,3720,6281,0,2314,9108,5134,6467,8066,7875,7319,9124,3946,6492


Notice that the dataframe above shows the distance between the different postal codes in meters to get a more accurate model when the distances between them are that small.

# Optimization Model

In [7]:
cost_warehouse = 100000000
km_cost=4 #Have not implemented the km-cost as  it needs to be an integer for the model to work. Then we loose a lot of accuracy
warehouse_cap = 10000000

#Demand for packages
num_packages_swiss_post=201000000 #https://www.post.ch/en/about-us/media/press-releases/2022/swiss-post-delivered-letters-and-parcels-to-customers-on-time-last-year
population_switzerland=8775000 #https://www.worldometers.info/world-population/switzerland-population/
market_share_amazon=0.08 #https://digital-commerce.post.ch/en/pages/blog/2020/online-marketplaces-in-switzerland
demand_per_habitant=num_packages_swiss_post*market_share_amazon/population_switzerland

demand=[int(int(string)*demand_per_habitant) for string in postal_codes['Population'].tolist()]

In [11]:
model = cp.CpModel()

# data
num_postal_codes = len(distances)
    
# declare variables
x = []
c = []
for i in range(num_postal_codes):
    x.append(model.NewBoolVar("x[%i]" % i))
    c.append([])
    for j in range(num_postal_codes):
        c[i].append(model.NewIntVar(0, warehouse_cap, "c[%i][%i]" % (i, j)))

z = model.NewIntVar(0, 999999999999, "z")

# objective to minimize.
model.Add(z == (sum(x) * cost_warehouse * 1000 + sum([c[i][j] * distances[i][j] * km_cost for i in range(num_postal_codes) for j in range(num_postal_codes)])))

# constraints
for j in range(num_postal_codes):
    model.Add(sum([c[i][j] for i in range(num_postal_codes)]) >= demand[j])
    
for i in range(num_postal_codes):
    model.Add(sum([c[i][j] for j in range(num_postal_codes)]) <= warehouse_cap*x[i])
    
for i in range(num_postal_codes):
    model.Add(sum([c[i][j] for j in range(num_postal_codes)]) >= 0)

model.Minimize(z)

# solution and search
solver = cp.CpSolver()
status = solver.Solve(model)

if status == cp.OPTIMAL:
    print("z:", solver.Value(z))
    print("x:", [solver.Value(x[i]) for i in range(num_postal_codes)])
    print("c:", [solver.Value(c[i][j]) for i in range(num_postal_codes) for j in range(num_postal_codes)])

print("NumConflicts:", solver.NumConflicts())
print("NumBranches:", solver.NumBranches())

z: 107780083152
x: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25826, 23547, 18905, 34060, 31284, 26017, 29156, 28887, 18561, 31142, 21137, 35539, 28669, 57424, 40963, 53310, 30067, 32841, 26094, 32596, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In the model above, the cost of each warehouse is multiplied by 1000. Because we calculate the distance in meters, we have to do this not to weigh the transport cost too high, i.e., have the same proportions between the cost of warehouses and transport cost.

In [9]:
#Display the postal code choosen as the optimal
warehouses = np.array([solver.Value(x[i]) for i in range(num_postal_codes)])
postal_codes[warehouses > 0][['Postal Code']]

,Postal Code
4,8005


As the cost minimized in our model is 1000 times as high as the actual cost, our cost is divided by 1000 in the textual description below to display the correct value.


In [10]:
postal_codes_list=postal_codes['Postal Code']

sent = np.array([solver.Value(c[i][j]) for i in range(num_postal_codes) for j in range(num_postal_codes)])
for i, val in enumerate(sent):
    if val > 0:
        from_i = i // num_postal_codes
        to_i = i% num_postal_codes
        print('from:',postal_codes_list[from_i],'to:',postal_codes_list[to_i], 'amount:', val, 'distance:', distances[from_i][to_i], 'price:', int(distances[from_i][to_i] * val/1000))

from: 8005 to: 8001 amount: 25826 distance: 2383 price: 61543
from: 8005 to: 8002 amount: 23547 distance: 3242 price: 76339
from: 8005 to: 8003 amount: 18905 distance: 1749 price: 33064
from: 8005 to: 8004 amount: 34060 distance: 1003 price: 34162
from: 8005 to: 8005 amount: 31284 distance: 0 price: 0
from: 8005 to: 8006 amount: 26017 distance: 2192 price: 57029
from: 8005 to: 8008 amount: 29156 distance: 4746 price: 138374
from: 8005 to: 8032 amount: 28887 distance: 4061 price: 117310
from: 8005 to: 8037 amount: 18561 distance: 1134 price: 21048
from: 8005 to: 8038 amount: 31142 distance: 5222 price: 162623
from: 8005 to: 8045 amount: 21137 distance: 3995 price: 84442
from: 8005 to: 8046 amount: 35539 distance: 3887 price: 138140
from: 8005 to: 8047 amount: 28669 distance: 3067 price: 87927
from: 8005 to: 8048 amount: 57424 distance: 3178 price: 182493
from: 8005 to: 8049 amount: 40963 distance: 3099 price: 126944
from: 8005 to: 8050 amount: 53310 distance: 3499 price: 186531
from: 80

By looking at https://www.worldpostalcodes.org/l1/en/ch/switzerland/map/r2/map-of-postalcodes-in-zurich we see the choice of 8005 as the optimal postal code makes sense as it is located in the city center.